In [1]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import pipeline

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel

import torch

In [2]:
torch_dtype = torch_dtype = torch.float16

In [3]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=False,
)

In [8]:
model = AutoModelForCausalLM.from_pretrained(
    'nijalmot/llama2-ko-dementia',
    quantization_config=quant_config,
    device_map={"": 0},
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
c:\Users\Harvey\anaconda3\envs\ml_dl\lib\site-packages\transformers\quantizers\auto.py:186: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at nijalmot/llama2-ko-dementia were not used when initializing LlamaForCausalLM: ['model.layers.0.self_attn.q_proj.base_layer.weight', 'model.layers.0.self_attn.q_proj.base_layer.weight.absmax', 'model.layers.0.self_attn.q_proj.base_layer.weight.quant_map', 'model.layers.0.self_attn.q_proj.base_layer.weight.quant_state.bitsandbytes__nf4', 'model.layers.0.self_attn.q_proj.lora_A.default.weight', 'model.layers.0.self_attn.q_proj.lora_B.default.weight', 'model.layers.0.self_attn.v_proj.base_layer.weight', 'model.layers.0.self_attn.v_proj.base_layer.weight.absmax', 'model.layers.0.self_attn.v_proj.base_layer.weight.quant_map', 'model.layers.0.self_attn.v_proj.base_layer.weight.quant_state.bitsandbytes__nf4', 'model.layers.0.self_attn.v_proj.lora_A.default.weight', 'model.layers.0.self_attn.v_proj.lora_B.default.weight', 'model.layers.1.self_attn.q_proj.base_layer.weight', 'model.layers.1.self_attn.q_proj.base_layer.weight.absmax', 'model.layers.1.self_a

In [9]:
tokenizer = AutoTokenizer.from_pretrained(
              'nijalmot/llama2-ko-dementia',
              trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [17]:
tokenizer.get_vocab()

{'▁ske': 18109,
 '▁맞음': 42620,
 '▁첩': 43415,
 '증권': 34666,
 '▁hand': 1361,
 '▁방송되는': 38919,
 '시스': 32972,
 '응': 45055,
 '▁partecip': 22885,
 '▁devient': 15210,
 '안이': 34334,
 '▁Sorry': 8221,
 '▁Click': 16297,
 '=""': 13776,
 '▁big': 4802,
 '▁nombre': 5419,
 '▁부끄럽': 39246,
 'nea': 14011,
 '人': 30313,
 'Reader': 6982,
 'hips': 14587,
 '▁다음': 32658,
 '▁_{': 7722,
 'qual': 15380,
 '▁corte': 28776,
 '필드': 44668,
 'gebaut': 27617,
 '▁уні': 25377,
 'жа': 2711,
 '▁막아': 36752,
 '▁Luis': 12583,
 '▁순': 32479,
 '▁카메': 36453,
 '▁참여를': 42480,
 '했다면': 40268,
 '▁debugger': 18297,
 '▁day': 2462,
 '▁갈등': 35610,
 '▁nave': 21102,
 'bum': 2404,
 'naio': 15616,
 '▁se': 409,
 '▁나라의': 36564,
 'ivent': 27649,
 'lef': 25874,
 '▁그린': 35895,
 'Organ': 27356,
 'aska': 16191,
 'pection': 27988,
 'fection': 20309,
 '▁kra': 18858,
 'ears': 15451,
 '쓴다': 38287,
 '▁accord': 15017,
 '▁few': 2846,
 'the': 1552,
 '▁wohl': 25304,
 '▁mate': 15358,
 '▁교통사고': 38648,
 '▁Gol': 20268,
 'ome': 608,
 '▁같냐': 41007,
 '▁struggling': 

In [22]:
import time

start = time.time()
prompt = "알츠하이머병을 집에서 치료하기 위해 집에서 실천할 수 있는 가장 효과적인 방법은 무엇인가요?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=300, truncation=True)
result = pipe(f"<s>[INST] {prompt} [/INST]")
now = time.time()
print(result[0]['generated_text'])
print(f"generate time : {now - start: .4f}")

<s>[INST] 알츠하이머병을 집에서 치료하기 위해 집에서 실천할 수 있는 가장 효과적인 방법은 무엇인가요? [/INST]
generate time :  21.7260
